# Example notebook

This is an example notebook that demonstrates how to use the bh24-literature-mining package to extract information from the literature.

In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
from bh24_literature_mining.utils import load_biotools_pub

tools = load_biotools_pub("../biotoolspub/biotoolspub.tsv")

for index, row in tools.head().iterrows():
    name = row["name"]
    pubmedid = row["pubmedid"]
    link = row["link"]
    print(f"Name: {name}, PubMed ID: {pubmedid}, Link: {link}")

Name: SubtiWiki, PubMed ID: PMC8728116, Link: fulltexts/PMC8728116.xml
Name: MetaPathways, PubMed ID: PMC4595896, Link: fulltexts/PMC4595896.xml
Name: iPHoP, PubMed ID: PMC10155999, Link: fulltexts/PMC10155999.xml
Name: MetaPathways, PubMed ID: PMC3695837, Link: fulltexts/PMC3695837.xml
Name: MZmine, PubMed ID: PMC1187873, Link: fulltexts/PMC1187873.xml


In [24]:
from bh24_literature_mining.europepmc_api import EuropePMCClient 


client = EuropePMCClient()
# Call bio.tools query and get a list of Article objects
biotools_articles = client.search_mentions("PeptideProphet")

first_article = biotools_articles[0]
print("The first title is:", first_article.title)
print("Type of the first article:", first_article.pubType)

# Call cites query with a specific PubMed ID and get a list of Article objects
cites_articles = client.search_cites("32109013")
print("The count of the cites articles (within the first 3 pages):", len(cites_articles))

The first title is: Reinvestigating the Correctness of Decoy-Based False Discovery Rate Control in Proteomics Tandem Mass Spectrometry.
Type of the first article: journal article
The count of the cites articles (within the first 3 pages): 100
